In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [0]:
DATADIR = '/content/drive/My Drive/UCMerced_LandUse/Images/'

CATEGORIES = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
img_rows = 256
img_cols = 256
num_class = 21
path = os.path.abspath('.cnn.py') #absolute path of program
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
X = []
Y = []
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')
dirs=dirs[:]
print(len(dirs))
'''

'''

In [0]:
label = 0
for i in dirs:
	n = 0
	count = 0
	for pic in glob.glob('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'+i+'/*.tif'):
		im = Image.open(pic)
		im = np.array(im)
		if((im.shape[0]==256) and (im.shape[1] ==256) and n<90): #get only 90 data
			r = im[:,:,0]
			g = im[:,:,1]
			b = im[:,:,2]
			X.append([r,g,b])
			Y.append([label])
			n = n + 1
	print(n)
	label = label + 1

In [0]:
import numpy as np
x_bck=X
y_bck=Y


X =  np.array(X)
Y =  np.array(Y)
X = X.reshape(X.shape[0], img_rows, img_cols, 3)
X.shape


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

input_shape = (img_rows, img_cols, 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = keras.utils.to_categorical(y_train, 21)
y_test = keras.utils.to_categorical(y_test, 21)

print(X_train.shape)
print(X_test.shape)

In [0]:
import time 
import cv2 
from scipy.fftpack import dct 

a=[]
b=[]
#y = np.random.random((5000, 6, 6, 3))
t1 = time.time()
a = np.zeros((len(X_train), 256, 256, 3))
b = np.zeros((len(X_test), 256, 256, 3))
for i in range(len(X_train)):
    for j in range(3):
        a[i, : , : , j] = cv2.dct(X_train[i, : , : , j])
        np.transpose(a, [0, 2, 1, 3])

for i in range(len(X_train)):
    for j in range(3):
        a[i, :, :, j] = cv2.dct(X_train[i, :, :, j]) 

for i in range(0,len(X_test)):
    for j in range(3):
        b[i, : , : , j] = cv2.dct(X_test[i, : , : , j])
        np.transpose(b, [0, 2, 1, 3])

for i in range(len(X_test)):
    for j in range(3):
        b[i, :, :, j] = cv2.dct(X_test[i, :, :, j]) 
 


N_ARY

In [0]:
a.shape

In [0]:
b.shape

In [0]:
a_cut = a[:,:64,:64,:]
b_cut = b[:,:64,:64,:]

In [0]:
print(a_cut.shape)
print(b_cut.shape)
a_cut[0]

In [0]:
a_ary = a_cut.reshape((1512,4096,3))
b_ary = b_cut.reshape((378,4096,3))

In [0]:
a_ary_bckp=a_ary
b_aray_bcp=b_ary

In [0]:
print(a_ary.shape)
a_ary[0]

In [0]:
final_a_ary = np.zeros((a_ary.shape[0],4098,3))
final_b_ary = np.zeros((b_ary.shape[0],4098,3))

In [0]:
n = 2                                              #Change n for specific n-ary (n = anything in 2*n)
mul=int(a_ary.shape[1])/n
avgs = []
count=0

for j in range(a_ary.shape[0]):
    avgR = []
    avgG = []
    avgB = []
    for i in range(n):
        l = int(mul*i)
        lend = int(mul*(i+1) - 1)
        np.sort(a_ary[j,:,0])
        np.sort(a_ary[j,:,1])
        np.sort(a_ary[j,:,2])
        avgR.append(np.average(a_ary[j,l:lend,0]))
        avgG.append(np.average(a_ary[j,l:lend,1]))
        avgB.append(np.average(a_ary[j,l:lend,2]))
    avgR = np.array(avgR)
    avgG = np.array(avgG)
    avgB = np.array(avgB)
    final_a_ary[j,:,0] = np.concatenate((a_ary[j,:,0],avgR))
    final_a_ary[j,:,1] = np.concatenate((a_ary[j,:,1],avgG))
    final_a_ary[j,:,2] = np.concatenate((a_ary[j,:,2],avgB))

mul=int(b_ary.shape[1])/n

for j in range(b_ary.shape[0]):
    avgR = []
    avgG = []
    avgB = []
    for i in range(n):
        l = int(mul*i)
        lend = int(mul*(i+1) - 1)
        np.sort(b_ary[j,:,0])
        np.sort(b_ary[j,:,1])
        np.sort(b_ary[j,:,2])
        avgR.append(np.average(b_ary[j,l:lend,0]))
        avgG.append(np.average(b_ary[j,l:lend,1]))
        avgB.append(np.average(b_ary[j,l:lend,2]))
    avgR = np.array(avgR)
    avgG = np.array(avgG)
    avgB = np.array(avgB)
    final_b_ary[j,:,0] = np.concatenate((b_ary[j,:,0],avgR))
    final_b_ary[j,:,1] = np.concatenate((b_ary[j,:,1],avgG))
    final_b_ary[j,:,2] = np.concatenate((b_ary[j,:,2],avgB))

In [0]:
final_a_ary.shape

In [0]:
final_b_ary.shape

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D, MaxPooling1D,Embedding
from keras.datasets import imdb
from keras.utils import plot_model
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
# Graphic output
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

**1DCNN ON MATRIX MIX**

In [0]:
model1dM = Sequential()
model1dM.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
model1dM.add(Conv1D(32, kernel_size=5))    # added new
model1dM.add(MaxPooling1D(pool_size=(2)))
model1dM.add(Conv1D(64, kernel_size=5))
model1dM.add(Conv1D(64, kernel_size=5))    # added new
model1dM.add(MaxPooling1D(pool_size=(2)))
model1dM.add(Conv1D(128, kernel_size=3))
model1dM.add(Conv1D(128, kernel_size=3))    # added new and changed to 128 from 64
model1dM.add(MaxPooling1D(pool_size=(2)))
model1dM.add(Dropout(0.25))
model1dM.add(Flatten())
model1dM.add(Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(l=0.1)))
model1dM.add(Dropout(0.5))
model1dM.add(Dense(21, activation='softmax'))
model1dM.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [0]:
model1dM.summary()

##With smaller model

In [0]:
model1dM.fit(final_a_ary, y_train, batch_size=50, nb_epoch=200, verbose=1, validation_data=(final_b_ary, y_test))

##With bigger modified model

In [0]:
model1dM.fit(final_a_ary, y_train, batch_size=50, nb_epoch=200, verbose=1, validation_data=(final_b_ary, y_test))

In [0]:
loss, acc = model1dM.evaluate(final_b_ary, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

**BATCH NORMALIZATION ON CONCATED MATRIX**

In [0]:
from keras.layers.normalization import BatchNormalization

# instantiate model
modelBNM = Sequential()
modelBNM.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
modelBNM.add(MaxPooling1D(pool_size=(2)))
modelBNM.add(Conv1D(32, kernel_size=5))
modelBNM.add(MaxPooling1D(pool_size=(2)))
modelBNM.add(Conv1D(64, kernel_size=3))
modelBNM.add(MaxPooling1D(pool_size=(2)))
modelBNM.add(Dropout(0.25))
modelBNM.add(Flatten())
# we can think of this chunk as the input layer
modelBNM.add(Dense(128,  init='uniform'))
modelBNM.add(BatchNormalization())
modelBNM.add(Activation('relu'))
modelBNM.add(Dropout(0.5))

# we can think of this chunk as the hidden layer    
modelBNM.add(Dense(64, init='uniform'))
modelBNM.add(BatchNormalization())
modelBNM.add(Activation('relu'))
modelBNM.add(Dropout(0.5))

# we can think of this chunk as the output layer
modelBNM.add(Dense(21, init='uniform'))
modelBNM.add(BatchNormalization())
modelBNM.add(Activation('softmax'))

# setting up the optimization of our weights 
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
modelBNM.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

# running the fitting


In [0]:
modelBNM.fit(final_a_ary, y_train, batch_size=50, nb_epoch=30, verbose=1, validation_data=(final_b_ary, y_test))

In [0]:
loss, acc = modelBNM.evaluate(final_b_ary, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

**BN with regularization on matrix concat** 

In [0]:
from keras.layers.normalization import BatchNormalization

# instantiate model
modelBNrm = Sequential()
modelBNrm.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
modelBNrm.add(MaxPooling1D(pool_size=(2)))
modelBNrm.add(Conv1D(32, kernel_size=5))
modelBNrm.add(MaxPooling1D(pool_size=(2)))
modelBNrm.add(Conv1D(64, kernel_size=3))
modelBNrm.add(MaxPooling1D(pool_size=(2)))
modelBNrm.add(Dropout(0.25))
modelBNrm.add(Flatten())
# we can think of this chunk as the input layer
modelBNrm.add(Dense(128,  init='uniform',kernel_regularizer=keras.regularizers.l2(l=0.1)))
modelBNrm.add(BatchNormalization())
modelBNrm.add(Activation('relu'))
modelBNrm.add(Dropout(0.5))

# we can think of this chunk as the hidden layer    
modelBNrm.add(Dense(64, init='uniform',kernel_regularizer=keras.regularizers.l2(l=0.1)))
modelBNrm.add(BatchNormalization())
modelBNrm.add(Activation('relu'))
modelBNrm.add(Dropout(0.5))

# we can think of this chunk as the output layer
modelBNrm.add(Dense(21, init='uniform',kernel_regularizer=keras.regularizers.l2(l=0.1)))
modelBNrm.add(BatchNormalization())
modelBNrm.add(Activation('softmax'))

# setting up the optimization of our weights 
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
modelBNrm.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

# running the fitting


In [0]:
modelBNrm.fit(final_a_ary, y_train, batch_size=50, nb_epoch=150, verbose=1, validation_data=(final_b_ary, y_test))

In [0]:
loss, acc = modelBNrm.evaluate(final_b_ary, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

BATCH NORMALIZATION ON CNN LAYERS WITH MATRIX

In [0]:
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.optimizers import SGD
# instantiate model
modelBNC = Sequential()
modelBNC.add(Conv1D(32, kernel_size=5, input_shape = (4098, 3)))
modelBNC.add(BatchNormalization())
modelBNC.add(MaxPooling1D(pool_size=(2)))
modelBNC.add(Conv1D(32, kernel_size=5))
modelBNC.add(BatchNormalization())
modelBNC.add(MaxPooling1D(pool_size=(2)))
modelBNC.add(Conv1D(64, kernel_size=3))
modelBNC.add(BatchNormalization())
modelBNC.add(MaxPooling1D(pool_size=(2)))
modelBNC.add(Dropout(0.25))
modelBNC.add(Flatten())
# we can think of this chunk as the input layer
modelBNC.add(Dense(128, init='uniform'))
modelBNC.add(BatchNormalization())
modelBNC.add(Activation('relu'))
modelBNC.add(Dropout(0.5))

# we can think of this chunk as the hidden layer    
modelBNC.add(Dense(64, init='uniform'))
modelBNC.add(BatchNormalization())
modelBNC.add(Activation('relu'))
modelBNC.add(Dropout(0.5))

# we can think of this chunk as the output layer
modelBNC.add(Dense(21, init='uniform'))
modelBNC.add(BatchNormalization())
modelBNC.add(Activation('softmax'))

# setting up the optimization of our weights 
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#modelBNC.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
modelBNC.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# running the fitting


In [0]:
modelBNC.fit(final_a_ary, y_train, batch_size=50, nb_epoch=30, verbose=1, validation_data=(final_b_ary, y_test))

In [0]:
loss, acc = modelBNC.evaluate(final_b_ary, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))